# Llama Stack RAG Lifecycle

In this notebook, we will walk through the lifecycle of building and evaluating a RAG pipeline using Llama Stack. 

**Example: Torchtune Knowledge Agent** 

Throughout this notebook, we will build a knowledge agent that can answer questions about the Torchtune project. 

## 0. Setup

In [2]:
from llama_stack_client import LlamaStackClient
from llama_stack.distribution.library_client import LlamaStackAsLibraryClient
from llama_stack_client.types.agent_create_params import AgentConfig
from llama_stack_client.lib.agents.agent import Agent
from rich.pretty import pprint
import json
import uuid
from pydantic import BaseModel
import rich
import os
try:
    from google.colab import userdata
    os.environ['FIREWORKS_API_KEY'] = userdata.get('FIREWORKS_API_KEY')
except ImportError:
    print("Not in Google Colab environment")

# client = LlamaStackAsLibraryClient("fireworks", provider_data = {"fireworks_api_key": os.environ['FIREWORKS_API_KEY']})
# _ = client.initialize()

# Uncomment to run on a hosted Llama Stack server
client = LlamaStackClient(base_url="http://localhost:8321")

MODEL_ID = "meta-llama/Llama-3.3-70B-Instruct"

Not in Google Colab environment


## 1. Simple Vanilla Agent

First, we will build a simple vanilla agent without any access to external knowledge base or tools, and check how it performs on a couple of questions. 


In [3]:
# First, let's come up with a couple of examples to test the agent
examples = [
    {
        "input_query": "What precision formats does torchtune support?",
        "expected_answer": "Torchtune supports two data types for precision: fp32 (full-precision) which uses 4 bytes per model and optimizer parameter, and bfloat16 (half-precision) which uses 2 bytes per model and optimizer parameter."
    },
    {
        "input_query": "What does DoRA stand for in torchtune?",
        "expected_answer": "Weight-Decomposed Low-Rank Adaptation"
    },
    {
        "input_query": "How does the CPUOffloadOptimizer reduce GPU memory usage?",
        "expected_answer": "The CPUOffloadOptimizer reduces GPU memory usage by keeping optimizer states on CPU and performing optimizer steps on CPU. It can also optionally offload gradients to CPU by using offload_gradients=True"
    },
    {
        "input_query": "How do I ensure only LoRA parameters are trainable when fine-tuning?",
        "expected_answer": "You can set only LoRA parameters to trainable using torchtune's utility functions: first fetch all LoRA parameters with lora_params = get_adapter_params(lora_model), then set them as trainable with set_trainable_params(lora_model, lora_params). The LoRA recipe handles this automatically."
    }
]

In [3]:
simple_agent = Agent(client, model=MODEL_ID, instructions="You are a helpful assistant that can answer questions about the Torchtune project.")
simple_session_id = simple_agent.create_session(session_name=f"simple_session_{uuid.uuid4()}")

In [4]:
for example in examples:
    response = simple_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": example["input_query"]
            }
        ],
        session_id=simple_session_id,
        stream=False
    )
    rich.print(f"[bold cyan]Question:[/bold cyan] {example['input_query']}")
    rich.print(f"[bold yellow]Agent Answer:[/bold yellow] {response.output_message.content}")

Question: What precision formats does torchtune support?

Agent Answer: Torchtune supports the following precision formats:

* `fp32` (32-bit floating point)
* `fp16` (16-bit floating point)
* `bf16` (Brain Floating Point 16, a 16-bit floating point format)
* `int8` (8-bit integer)

These precision formats can be used for model weights, activations, and gradients, allowing for flexible 
mixed-precision training and inference.

Question: What does DoRA stand for in torchtune?

Agent Answer: In Torchtune, DoRA stands for "Dynamic Output Range Awareness"

Question: How does the CPUOffloadOptimizer reduce GPU memory usage?

Agent Answer: The CPUOffloadOptimizer in Torchtune reduces GPU memory usage by offloading the model's parameters 
and gradients to the CPU during the backward pass, freeing up GPU memory. This allows for training larger models 
that would otherwise not fit in GPU memory.

Question: How do I ensure only LoRA parameters are trainable when fine-tuning?

Agent Answer: To ensure only LoRA (Low-Rank Adaptation) parameters are trainable when fine-tuning, you can set the 
`requires_grad` attribute of the original model parameters to `False` and the `requires_grad` attribute of the LoRA
parameters to `True`. This will freeze the original model weights and only update the LoRA parameters during 
fine-tuning.

#### 1.1 Evaluate Agent Responses
Let's gather up the agent's logs and evaluate the agent's performance. We can see that our agent's response is quite bad and off from the expected answer.

In [26]:
eval_rows = []
session_response = client.agents.session.retrieve(agent_id=simple_agent.agent_id, session_id=simple_session_id)
for i, turn in enumerate(session_response.turns):
    eval_rows.append({
        "input_query": examples[i]["input_query"],
        "expected_answer": examples[i]["expected_answer"],
        "generated_answer": turn.output_message.content,
    })

scoring_params = {
    "braintrust::factuality": None,
}
scoring_response = client.scoring.score(
    input_rows=eval_rows,
    scoring_functions=scoring_params,
)
pprint(scoring_response)

ScoringScoreResponse(
│   results={
│   │   'braintrust::factuality': ScoringResult(
│   │   │   aggregated_results={'average': {'average': 0.0}},
│   │   │   score_rows=[
│   │   │   │   {
│   │   │   │   │   'score': 0.0,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'D',
│   │   │   │   │   │   'rationale': "1. The expert answer states that Torchtune supports two precision formats: fp32 (full-precision) and bfloat16 (half-precision).\n2. The submitted answer lists three precision formats: Full precision (FP32), Mixed precision (FP16), and Integer precision (INT8).\n3. The submitted answer mentions FP32, which aligns with the expert answer's mention of fp32.\n4. The submitted answer does not mention bfloat16, which is included in the expert answer.\n5. The submitted answer includes FP16 and INT8, which are not mentioned in the expert answer.\n6. Since the submitted answer includes precision formats not mentioned by the expert and omits bfloat16, there is a disagreement between the two answers regarding the precision formats supported by Torchtune.\n\nBased on this analysis, the correct choice is (D) There is a disagreement between the submitted answer and the expert answer."
│   │   │   │   │   }
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 0.0,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'D',
│   │   │   │   │   │   'rationale': '1. The expert answer states that DoRA stands for "Weight-Decomposed Low-Rank Adaptation."\n2. The submitted answer states that DoRA stands for "Deep Optimization and Rapid Auto-tuning."\n3. The two answers provide completely different expansions for the acronym DoRA.\n4. Since the expansions are different, there is a clear disagreement between the submitted answer and the expert answer regarding what DoRA stands for in torchtune.\n5. Therefore, the submitted answer conflicts with the expert answer.'
│   │   │   │   │   }
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 0.0,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'D',
│   │   │   │   │   │   'rationale': "1. **Expert Answer**: The expert states that the CPUOffloadOptimizer reduces GPU memory usage by keeping optimizer states on the CPU and performing optimizer steps on the CPU. It also mentions the optional offloading of gradients to the CPU.\n\n2. **Submitted Answer**: The submission describes the CPUOffloadOptimizer as offloading certain model parameters and intermediate results to the CPU, which is different from the expert's focus on optimizer states and steps. The submission does not mention optimizer states or steps but instead focuses on parameters and intermediate results.\n\n3. **Comparison**:\n   - The expert's answer focuses on optimizer states and steps being offloaded to the CPU.\n   - The submission focuses on model parameters and intermediate results being offloaded to the CPU.\n   - The submission does not mention the offloading of gradients or optimizer states and steps.\n\n4. **Conclusion**: There is a disagreement between the submitted answer and the expert answer. The submission describes a different mechanism (offloading model parameters and intermediate results) than the expert (offloading optimizer states and steps). Therefore, the correct choice is (D) There is a disagreement between the submitted answer and the expert answer."
│   │   │   │   │   }
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 0.0,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'D',
│   │   │   │   │   │   'rationale': "1. **Expert Answer Analysis**: The expert answer suggests using torchtune's utility functions to set only LoRA parameters as trainable. It involves fetching LoRA parameters using `get_adapter_params(lora_model)` and then setting them as trainable with `set_trainable_params(lora_model, lora_params)`. It also mentions that the LoRA recipe handles this automatically.\n\n2. **Submitted Answer An

## 2. Search Agent

Now, let's see how we can improve the agent's performance by adding a search tool.

In [5]:
search_agent = Agent(client, 
                     model=MODEL_ID,
                     instructions="You are a helpful assistant that can answer questions about the Torchtune project. You should always use the search tool to answer questions.",
                     tools=["builtin::websearch"])
search_session_id = search_agent.create_session(session_name=f"search_session_{uuid.uuid4()}")

In [6]:
for example in examples:
    response = search_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": example["input_query"]
            }
        ],
        session_id=search_session_id,
        stream=False
    )
    rich.print(f"[bold cyan]Question:[/bold cyan] {example['input_query']}")
    rich.print(f"[bold yellow]Agent Answer:[/bold yellow] {response.output_message.content}")

Question: What precision formats does torchtune support?

Agent Answer: Torchtune supports the following precision formats:

* bf16 (16-bit floating-point format)
* fp32 (32-bit floating-point format, also known as "full-precision")

It's worth noting that torchtune also supports mixed-precision techniques, which allow for the use of different 
precision formats for different parts of the model or during different stages of training.

Question: What does DoRA stand for in torchtune?

Agent Answer: DoRA stands for "Decoupled Offline Reweighting Adapter" in torchtune. It is a technique used for 
fine-tuning large language models (LLMs) and is an alternative to LoRA (Low-Rank Adaptation). DoRA is designed to 
be more efficient and effective than LoRA, and it has been shown to achieve high performance on various fine-tuning
tasks.

Question: How does the CPUOffloadOptimizer reduce GPU memory usage?

Agent Answer: The CPUOffloadOptimizer reduces GPU memory usage by offloading gradients and optimizer states to the 
CPU, freeing up memory on the GPU. This is done by setting `offload_gradients=True` in the CPUOffloadOptimizer, 
which transfers gradients from the GPU to the CPU once the device-to-host transfer finishes. Additionally, using a 
stateful optimizer with a model with a lot of parameters and not using gradient accumulation can also help minimize
the slowdown caused by offloading.

Question: How do I ensure only LoRA parameters are trainable when fine-tuning?

Agent Answer: To ensure only LoRA parameters are trainable when fine-tuning, you can use the `LoRA` class from the 
`transformers` library and set the `trainable` parameter to `True` only for the LoRA parameters. You can also use 
the `freeze` method to freeze the weights of the pre-trained model and only update the LoRA parameters during 
fine-tuning.

Here is an example code snippet:
```
from transformers import AutoModelForSequenceClassification, LoRA

# Load pre-trained model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Create LoRA adapter
lora = LoRA(model, rank=16)

# Freeze pre-trained model weights
for param in model.parameters():
    param.requires_grad = False

# Set LoRA parameters to trainable
for param in lora.parameters():
    param.requires_grad = True

# Fine-tune model with LoRA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(lora.parameters(), lr=1e-5)

for epoch in range(5):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_dataloader)}")
```
In this example, we first load a pre-trained BERT model and create a LoRA adapter with a rank of 16. We then freeze
the pre-trained model weights and set the LoRA parameters to trainable. Finally, we fine-tune the model with the 
LoRA adapter using the AdamW optimizer and cross-entropy loss.

#### 2.1 Evaluate Agent Responses

We can see that with a search tool, the agent's performance is much better, and have less hallucinations. 

In [7]:
eval_rows = []
session_response = client.agents.session.retrieve(agent_id=search_agent.agent_id, session_id=search_session_id)
for i, turn in enumerate(session_response.turns):
    eval_rows.append({
        "input_query": examples[i]["input_query"],
        "expected_answer": examples[i]["expected_answer"],
        "generated_answer": turn.output_message.content,
    })

scoring_params = {
    "braintrust::factuality": None,
}
scoring_response = client.scoring.score(
    input_rows=eval_rows,
    scoring_functions=scoring_params,
)
pprint(scoring_response)

ScoringScoreResponse(
│   results={
│   │   'braintrust::factuality': ScoringResult(
│   │   │   aggregated_results={'average': {'average': 0.44999999999999996}},
│   │   │   score_rows=[
│   │   │   │   {
│   │   │   │   │   'score': 0.6,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'B',
│   │   │   │   │   │   'rationale': '1. **Expert Answer Analysis**: The expert answer states that Torchtune supports two precision formats: fp32 (full-precision) and bfloat16 (half-precision).\n   - fp32 is described as using 4 bytes per model and optimizer parameter.\n   - bfloat16 is described as using 2 bytes per model and optimizer parameter.\n\n2. **Submitted Answer Analysis**: The submitted answer lists the following precision formats:\n   - bf16 (16-bit floating-point format)\n   - fp32 (32-bit floating-point format, also known as "full-precision")\n   - Additionally, it mentions support for mixed-precision techniques.\n\n3. **Comparison**:\n   - Both answers mention fp32 and bf16/bfloat16, which are essentially the same formats, though the naming differs slightly (bf16 vs bfloat16).\n   - The submitted answer includes additional information about mixed-precision techniques, which is not mentioned in the expert answer.\n\n4. **Conclusion**:\n   - The submitted answer includes all the information from the expert answer and adds more details about mixed-precision techniques.\n   - There is no conflict between the two answers; the submitted answer is more detailed.\n\nTherefore, the submitted answer is a superset of the expert answer and is fully consistent with it.'
│   │   │   │   │   }
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 0.0,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'D',
│   │   │   │   │   │   'rationale': '1. The expert answer states that DoRA stands for "Weight-Decomposed Low-Rank Adaptation."\n2. The submitted answer states that DoRA stands for "Decoupled Offline Reweighting Adapter."\n3. The two answers provide completely different expansions for the acronym DoRA.\n4. Since the expansions are different, there is a clear disagreement between the submitted answer and the expert answer regarding what DoRA stands for.\n5. Therefore, the correct choice is that there is a disagreement between the submitted answer and the expert answer.'
│   │   │   │   │   }
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 0.6,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'B',
│   │   │   │   │   │   'rationale': "1. **Expert Answer Analysis**: The expert answer states that the CPUOffloadOptimizer reduces GPU memory usage by keeping optimizer states on the CPU and performing optimizer steps on the CPU. It also mentions the optional offloading of gradients to the CPU by setting `offload_gradients=True`.\n\n2. **Submitted Answer Analysis**: The submitted answer mentions offloading both gradients and optimizer states to the CPU, which aligns with the expert's mention of keeping optimizer states on the CPU and optionally offloading gradients. It also adds details about freeing up GPU memory, the process of transferring gradients once the device-to-host transfer finishes, and additional strategies like using a stateful optimizer and not using gradient accumulation to minimize slowdown.\n\n3. **Comparison**: The submitted answer includes all the points mentioned in the expert answer:\n   - Offloading optimizer states to the CPU.\n   - Optionally offloading gradients to the CPU with `offload_gradients=True`.\n   - It adds more details about the process and additional strategies to minimize slowdown, which are not mentioned in the expert answer.\n\n4. **Conclusion**: The submitted answer is a superset of the expert answer as it contains all the information from the expert answer and adds more details. Therefore, the correct choice is (B)."
│   │   │   │   │   }
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 0.6,
│   │   │  

### 3. RAG Agent

Now, let's see how we can improve the agent's performance by adding a RAG tool that explicitly retrieves information from Torchtune's documentation. 

In [4]:
from llama_stack_client.types import Document
urls = [
    "memory_optimizations.rst",
    "chat.rst",
    "llama3.rst",
    "datasets.rst",
    "qat_finetune.rst",
    "lora_finetune.rst",
]
documents = [
    Document(
        document_id=f"num-{i}",
        content=f"https://raw.githubusercontent.com/pytorch/torchtune/main/docs/source/tutorials/{url}",
        mime_type="text/plain",
        metadata={},
    )
    for i, url in enumerate(urls)
]

vector_providers = [
    provider for provider in client.providers.list() if provider.api == "vector_io"
]
selected_vector_provider = vector_providers[0]
vector_db_id = f"test_vector_db_{uuid.uuid4()}"
client.vector_dbs.register(
    vector_db_id=vector_db_id,
    embedding_model="all-MiniLM-L6-v2",
    embedding_dimension=384,
    provider_id=selected_vector_provider.provider_id,
)

client.tool_runtime.rag_tool.insert(
    documents=documents,
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=512,
)

In [7]:
rag_agent = Agent(
    client,
    model=MODEL_ID,
    instructions="You are a helpful assistant that can answer questions about the Torchtune project. You should always use the RAG tool to answer questions.",
    tools=[{
        "name": "builtin::rag",
        "args": {"vector_db_ids": [vector_db_id]},
    }],
)

rag_session_id = rag_agent.create_session(session_name=f"rag_session_{uuid.uuid4()}")
for example in examples:
    response = rag_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": example["input_query"]
            }
        ],
        session_id=rag_session_id,
        stream=False
    )
    rich.print(f"[bold cyan]Question:[/bold cyan] {example['input_query']}")
    rich.print(f"[bold yellow]Agent Answer:[/bold yellow] {response.output_message.content}")

Question: What precision formats does torchtune support?

Agent Answer: Torchtune supports the following precision formats: 

1. fp32 (full precision, 4 bytes per model and optimizer parameter)
2. bfloat16 (half-precision, 2 bytes per model and optimizer parameter)
3. int8 (integer 8-bit, used for quantized models)
4. int4 (integer 4-bit, used for quantized models)

Question: What does DoRA stand for in torchtune?

Agent Answer: DoRA stands for "Decoupled Offline Replay Alignment" in torchtune.

Question: How does the CPUOffloadOptimizer reduce GPU memory usage?

Agent Answer: The CPUOffloadOptimizer reduces GPU memory usage by offloading optimizer states and gradients to CPU,
and performing optimizer steps on CPU. This can significantly reduce GPU memory usage at the cost of CPU RAM and 
training speed. It is recommended to use this optimizer only if other techniques are not enough. Additionally, it 
is suggested to use full bf16 training to minimize the slowdown and give GPU more work per optimizer step to 
amortize the offloading time.

Question: How do I ensure only LoRA parameters are trainable when fine-tuning?

Agent Answer: To ensure only LoRA parameters are trainable when fine-tuning, you can use the `set_trainable_params`
function from `torchtune.modules.peft.peft_utils` to set `requires_grad=True` on LoRA parameters and 
`requires_grad=False` on all other parameters. This can be done after loading the base model weights into the LoRA 
model. Additionally, you can use the `get_adapter_params` function to fetch all parameters associated with LoRA.

In [8]:
eval_rows = []
session_response = client.agents.session.retrieve(agent_id=rag_agent.agent_id, session_id=rag_session_id)
for i, turn in enumerate(session_response.turns):
    eval_rows.append({
        "input_query": examples[i]["input_query"],
        "expected_answer": examples[i]["expected_answer"],
        "generated_answer": turn.output_message.content,
    })

scoring_params = {
    "braintrust::factuality": None,
}
scoring_response = client.scoring.score(
    input_rows=eval_rows,
    scoring_functions=scoring_params,
)
pprint(scoring_response)

ScoringScoreResponse(
│   results={
│   │   'braintrust::factuality': ScoringResult(
│   │   │   aggregated_results={'average': {'average': 0.44999999999999996}},
│   │   │   score_rows=[
│   │   │   │   {
│   │   │   │   │   'score': 0.6,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'B',
│   │   │   │   │   │   'rationale': "1. The expert answer states that Torchtune supports two precision formats: fp32 and bfloat16.\n2. The submitted answer includes the same two precision formats: fp32 and bfloat16, with the same descriptions regarding their byte usage.\n3. Additionally, the submitted answer mentions two more precision formats: int8 and int4, which are used for quantized models.\n4. Since the submitted answer includes all the information from the expert answer and adds more details (int8 and int4), it is a superset of the expert answer.\n5. There is no conflict between the submitted answer and the expert answer; the additional information does not contradict the expert's information.\n6. Therefore, the submitted answer is a superset of the expert answer and is fully consistent with it."
│   │   │   │   │   }
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 0.0,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'D',
│   │   │   │   │   │   'rationale': '1. The expert answer states that DoRA stands for "Weight-Decomposed Low-Rank Adaptation."\n2. The submitted answer states that DoRA stands for "Decoupled Offline Replay Alignment."\n3. The two answers provide completely different expansions for the acronym DoRA.\n4. Since the expansions are different, there is a clear disagreement between the submitted answer and the expert answer regarding what DoRA stands for in torchtune.\n5. Therefore, the correct choice is (D) There is a disagreement between the submitted answer and the expert answer.'
│   │   │   │   │   }
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 0.6,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'B',
│   │   │   │   │   │   'rationale': "1. **Expert Answer Content**: The expert answer states that the CPUOffloadOptimizer reduces GPU memory usage by keeping optimizer states on the CPU and performing optimizer steps on the CPU. It also mentions the optional offloading of gradients to the CPU using `offload_gradients=True`.\n\n2. **Submitted Answer Content**: The submitted answer includes all the points mentioned in the expert answer: offloading optimizer states and gradients to the CPU, and performing optimizer steps on the CPU. Additionally, it provides more context by discussing the trade-offs involved, such as increased CPU RAM usage and potential training speed reduction. It also suggests using full bf16 training to mitigate these issues.\n\n3. **Comparison**: The submitted answer contains all the factual elements of the expert answer and adds more information about the implications and recommendations for using the CPUOffloadOptimizer. There is no conflict between the two answers; rather, the submission expands on the expert's points.\n\n4. **Conclusion**: Since the submitted answer includes all the details from the expert answer and adds additional relevant information, it is a superset of the expert answer and is fully consistent with it.\n\nTherefore, the correct choice is (B)."
│   │   │   │   │   }
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 0.6,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'B',
│   │   │   │   │   │   'rationale': "1. **Expert Answer Analysis**: The expert answer provides a method to ensure only LoRA parameters are trainable by using torchtune's utility functions. It mentions fetching LoRA parameters with `get_adapter_params(lora_model)` and setting them as trainable with `set_trainable_params(lora_model, lora_params)`. It also notes that the LoRA recipe handles this automatically.\n\n2. **Submitted Answer Analysis**: The submitted answer also desc

#### Deep dive into RAG Tool Performance
- Now, let's take a closer look at how the RAG tool is doing, specifically on the second example where the agent's answer is not correct on identifying what DoRA stands for. 
- Notice that the issue lies with the retrieval step, where the retrieved document is not relevant to the question. 

In [11]:
session_response = client.agents.session.retrieve(agent_id=rag_agent.agent_id, session_id=rag_session_id)
pprint(session_response.turns[1])

Turn(
│   input_messages=[UserMessage(content='What does DoRA stand for in torchtune?', role='user', context=None)],
│   output_message=CompletionMessage(
│   │   content='DoRA stands for "Decoupled Offline Replay Alignment" in torchtune.',
│   │   role='assistant',
│   │   stop_reason='end_of_turn',
│   │   tool_calls=[]
│   ),
│   session_id='b858ba10-8d18-4e1f-b7d0-5684daff0e71',
│   started_at=datetime.datetime(2025, 3, 6, 15, 45, 27, 129881, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600))),
│   steps=[
│   │   InferenceStep(
│   │   │   api_model_response=CompletionMessage(
│   │   │   │   content='',
│   │   │   │   role='assistant',
│   │   │   │   stop_reason='end_of_turn',
│   │   │   │   tool_calls=[
│   │   │   │   │   ToolCall(
│   │   │   │   │   │   arguments={'query': 'DoRA meaning in torchtune'},
│   │   │   │   │   │   call_id='abc4bb5c-ecc5-42a9-a604-c7a5b76220a9',
│   │   │   │   │   │   tool_name='knowledge_search'
│   │   │   │   │   )
│   │   │   │   ]
│   │   │   ),
│   │   │   step_id='c50c84e6-5ca7-4aa7-8c53-e01d639275eb',
│   │   │   step_type='inference',
│   │   │   turn_id='169a465d-9f29-43aa-9c70-e41393a9a504',
│   │   │   completed_at=datetime.datetime(2025, 3, 6, 15, 45, 27, 900249, tzinfo=TzInfo(-08:00)),
│   │   │   started_at=datetime.datetime(2025, 3, 6, 15, 45, 27, 130048, tzinfo=TzInfo(-08:00))
│   │   ),
│   │   ToolExecutionStep(
│   │   │   step_id='c1f98a74-0837-4f6d-9e70-7ec6b12d6413',
│   │   │   step_type='tool_execution',
│   │   │   tool_calls=[
│   │   │   │   ToolCall(
│   │   │   │   │   arguments={'query': 'DoRA meaning in torchtune'},
│   │   │   │   │   call_id='abc4bb5c-ecc5-42a9-a604-c7a5b76220a9',
│   │   │   │   │   tool_name='knowledge_search'
│   │   │   │   )
│   │   │   ],
│   │   │   tool_responses=[
│   │   │   │   ToolResponse(
│   │   │   │   │   call_id='abc4bb5c-ecc5-42a9-a604-c7a5b76220a9',
│   │   │   │   │   content=[
│   │   │   │   │   │   TextContentItem(
│   │   │   │   │   │   │   text='knowledge_search tool found 5 chunks:\nBEGIN of knowledge_search tool results.\n',
│   │   │   │   │   │   │   type='text'
│   │   │   │   │   │   ),
│   │   │   │   │   │   TextContentItem(
│   │   │   │   │   │   │   text='Result 1:\nDocument_id:num-0\nContent: etune\n:func:`torchtune.models.llama3.llama3_8b` with DoRA, you would use :func:`torchtune.models.llama3.lora_llama3_8b` with ``use_dora=True``:\n\n.. code-block:: bash\n\n  tune run lora_finetune_single_device --config llama3/8B_lora_single_device \\\n  model.use_dora=True\n\n.. code-block:: yaml\n\n  model:\n    _component_: torchtune.models.lora_llama3_8b\n    use_dora: True\n\nSince DoRA extends LoRA, the parameters for :ref:`customizing LoRA <glossary_lora>` are identical. You can also quantize the base model weights like in :ref:`glossary_qlora` by using ``quantize=True`` to reap\neven more memory savings!\n\n.. code-block:: bash\n\n  tune run lora_finetune_single_device --config llama3/8B_lora_single_device \\\n  model.apply_lora_to_mlp=True \\\n  model.lora_attn_modules=["q_proj","k_proj","v_proj"] \\\n  model.lora_rank=16 \\\n  model.lora_alpha=32 \\\n  model.use_dora=True \\\n  model.quantize_base=True\n\n.. code-block:: yaml\n\n  model:\n    _component_: torchtune.models.lora_llama3_8b\n    apply_lora_to_mlp: True\n    lora_attn_modules: ["q_proj", "k_proj", "v_proj"]\n    lora_rank: 16\n    lora_alpha: 32\n    use_dora: True\n    quantize_base: True\n\n\n.. note::\n\n   Under the hood, we\'ve enabled DoRA by adding the :class:`~torchtune.modules.peft.DoRALinear` module, which we swap\n   out for :class:`~torchtune.modules.peft.LoRALinear` when ``use_dora=True``.\n\n.. _glossary_distrib:\n\n\n.. TODO\n\n.. Distributed\n.. -----------\n\n.. .. _glossary_fsdp:\n\n.. Fully Sharded Data Parallel (FSDP)\n.. ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n\n.. All our ``_distributed`` recipes use `FSDP <https://pytorch.org/docs/stable/fsdp.html>`.\n.. .. _glossary_fsdp2:\n\n',
│   │   │   │   │   │